
# Basic NLP operations

1. Lemmatization
1. Part-of-speech (POS) tagging
1. Dependency parsing
1. Rule-based pattern matching
1. Named entity recognition (NER)
1. Word embeddings

## Lemmatization

In [1]:
import spacy

nlp = spacy.load('en_core_web_sm')

In [2]:
text = u'''She goes for a walk every day.
           He was going to the cinema in the evening.
           They have already gone to work.
           I took my coffee to go and went home.'''
word_to_find = u'go'

In [3]:
doc = nlp(text)

for token in doc:
    if token.lemma_ == word_to_find:
        print(f'{token.text:<8} {token.lemma_}')

goes     go
going    go
gone     go
go       go
went     go


## Part-of-speech (POS) tagging

In [4]:
text = u'The sky above the port was the color of television, tuned to a dead channel.'

In [5]:
doc = nlp(text)

for token in doc:
    print(f'{token.text:<12} {token.pos_}')

The          DET
sky          NOUN
above        ADP
the          DET
port         NOUN
was          VERB
the          DET
color        NOUN
of           ADP
television   NOUN
,            PUNCT
tuned        VERB
to           ADP
a            DET
dead         ADJ
channel      NOUN
.            PUNCT


## Dependency parsing

In [6]:
text = u'Bob never took chemistry in school.'
doc = nlp(text)

In [7]:
from spacy import displacy

displacy.render(doc, style="dep", jupyter=True)

## Rule-based pattern matching

In [8]:
text = u'''Corrective actions to the previous audit findings are not implemented in a timely manner.
           Most are not fully documented.
           There are even some findings that the team never discussed.'''
doc = nlp(text)

In [9]:
from spacy.matcher import Matcher

negated_verb_pattern = [ {'DEP': 'neg'}, {'POS': 'VERB'} ]

matcher = Matcher(nlp.vocab)
matcher.add('NEGATED_VERB', None, negated_verb_pattern)
matches = matcher(doc)

for rule_id, start_token, end_token in matches:
    print(doc[start_token:end_token])

not implemented
never discussed


In [10]:
# {'DEP': 'advmod', 'OP': '?'}, 
# advmod - adverbial modifier

In [11]:
# Token attributes - https://spacy.io/usage/linguistic-features#adding-patterns-attributes
#    DEP - syntactic dependency
#    OP  - quantifier (`?` means optional)

# Dependency tokens - https://stackoverflow.com/a/40288324/95
#    neg    - negation modifier
#    advmod - adverbial modifier

## Named entity recognition (NER)

In [12]:
text = u'Marek Grzenkowicz came to the Devoxx conference from Poland yesterday around 11 AM.'
doc = nlp(text)

In [13]:
from spacy import displacy

displacy.render(doc, style="ent", jupyter=True)

In [14]:
# Default spaCy entity types - https://spacy.io/api/annotation#named-entities

## Word embeddings

In [15]:
# larger model, with word vectors
nlp = spacy.load('en_core_web_md')

1. Context-free language models
   - word2vec
   - **GloVe** &nbsp;&nbsp;&nbsp;&nbsp; ← used by default spaCy models
   - fastText
1. Contextual language models
   - ELMo
   - BERT

1. Context-free language models - vectors trained on a text corpus from co-occurrence statistics
1. Contextual models - word representations that are a function of the entire context of sentence or paragraph

## Word embeddings

In [16]:
vector_dog = nlp.vocab[u'dog'].vector
print(vector_dog[:42])

[-0.40176   0.37057   0.021281 -0.34125   0.049538  0.2944   -0.17376
 -0.27982   0.067622  2.1693   -0.62691   0.29106  -0.6727    0.23319
 -0.34264   0.18311   0.50226   1.0689    0.14698  -0.4523   -0.41827
 -0.15967   0.26748  -0.48867   0.36462  -0.043403 -0.24474  -0.41752
  0.089088 -0.25552  -0.55695   0.12243  -0.083526  0.55095   0.3641
  0.15361   0.55738  -0.90702  -0.049098  0.3858    0.38      0.14425 ]


In [17]:
print(f'length  = {len(vector_dog)}')
print(f'min_val = {min(vector_dog)}')
print(f'max_val = {max(vector_dog)}')

length  = 300
min_val = -2.7049999237060547
max_val = 2.169300079345703


## Comparing word vectors

**Cosine similarity**
![cosine similarity in 2D](./images/vectors_cos_sim_600.png)

**Note:** For illustrative purposes only; the values above are random.

## Comparing word vectors

In [18]:
from sklearn.metrics.pairwise import cosine_similarity

words = ['dog', 'husky', 'cat', 'horse', 'tree', 'stone', 'bitcoin']

for word in words:
    vector_word = nlp.vocab[word].vector
    
    # calculate cosine between two vectors
    cos_sim = cosine_similarity([vector_dog], [vector_word])
    
    cos_sim = cos_sim[0][0]
    print(f'cos(dog, {word + ")":<8} = {cos_sim:>6.3f}')

cos(dog, dog)     =  1.000
cos(dog, husky)   =  0.859
cos(dog, cat)     =  0.802
cos(dog, horse)   =  0.625
cos(dog, tree)    =  0.326
cos(dog, stone)   =  0.265
cos(dog, bitcoin) = -0.085


## Arithmetic of word vectors

In [19]:
from sklearn.metrics.pairwise import cosine_similarity

def find_similar_vectors(word_vector, vocabulary, skip_words=(), num=3):
    """
    Finds vectors close to `word_vector` in terms of cosine similarity,
    inside given `vocabulary`.
    """
    result = [
        w
        for w in vocabulary
        if w.prob >= -15
        and w.is_lower
        and any(w.vector != word_vector)
        and w.orth_ not in skip_words
    ]
    result = sorted(
        result,
        key=lambda w: cosine_similarity([w.vector], [word_vector]),
        reverse=True
    )
    return [w.orth_ for w in result[:num]]

## Arithmetic of word vectors

In [20]:
find_similar_vectors(vector_dog, nlp.vocab, num=6)

['canines', 'dogs', 'puppy', 'poodle', 'terrier', 'husky']

In [21]:
vector_spring = nlp.vocab[u'spring'].vector
find_similar_vectors(vector_spring, nlp.vocab, num=6)

['summertime', 'summer', 'autumn', 'winter', 'fall', 'sprung']

![spring ambiguity](./images/springs.jpg)

Photo by Anisur Rahman on Unsplash (season)

Photo by Alexander Schimmeck on Unsplash (metal coil)

## Arithmetic of word vectors
\begin{equation}
\LARGE{\mathit{    v_{parent} + v_{woman} = x    }}
\end{equation}

In [22]:
x = nlp.vocab[u'parent'].vector + nlp.vocab[u'woman'].vector
find_similar_vectors(x, nlp.vocab, skip_words=[u'parent', u'woman'])

['mother', 'child', 'girl']

\begin{equation}
\LARGE{\mathit{    v_{seawater} - v_{salt} = x    }}
\end{equation}

In [23]:
x = nlp.vocab[u'seawater'].vector - nlp.vocab[u'salt'].vector
find_similar_vectors(x, nlp.vocab, skip_words=[u'seawater', u'salt'])

['water', 'air', 'hydrant']

## Arithmetic of word vectors
\begin{equation}
\LARGE{\mathit{    v_{Switzerland} - v_{Basel} = x - v_{Poznan}    }}
\end{equation}

\begin{equation}
\large{\mathit{    x = v_{Poznan} + (v_{Switzerland} - v_{Basel})    }}
\end{equation}

In [24]:
new_concept = nlp.vocab[u'switzerland'].vector - nlp.vocab[u'basel'].vector
x = nlp.vocab[u'poznan'].vector + new_concept
find_similar_vectors(x, nlp.vocab, skip_words=[u'poznan', u'switzerland', u'basel'])

['poland', 'sweden', 'norway']

\begin{equation}
\large{\mathit{    v_{Poland} = 
                     \underbrace{v_{Poznan}}_\text{a city}
                   + \underbrace{(v_{Switzerland} - v_{Basel})}_\text{concept of city's country}
}}
\end{equation}

[![Built with spaCy](https://img.shields.io/badge/built%20with-spaCy-09a3d5.svg)](https://spacy.io)